<a href="https://colab.research.google.com/github/phhduc/testweb.github.io/blob/main/server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask flask_restful pyngrok surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 8.5 MB/s 
     |████████████████████████████████| 52 kB 1.5 MB/s 
     |████████████████████████████████| 11.8 MB 28.3 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=1e11c3cd73b18ea933fa3d323d666c4bb5680ebd09636c8dd863ce385819f217
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633735 sha256=71b1e2ae867f459130a50fc89cdcb7c47d2857517c7fd286a1df71b0302b8113
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built pyngrok scikit-surprise


In [2]:
import flask 
import flask_restful
from pyngrok import ngrok
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
import json
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.prediction_algorithms import knns
from surprise.model_selection import train_test_split, KFold, GridSearchCV
from surprise.similarities import cosine, pearson
from surprise import accuracy
from surprise import dump
import os
import random
from collections import defaultdict
from time import time

In [4]:
ratings = pd.read_csv("/content/drive/MyDrive/ai/data/interactions_x.csv")

In [3]:
f=open("/content/drive/MyDrive/ai/data/books_titles.json")
titles_dict = json.load(f)
f.close()

In [6]:
titles = pd.DataFrame.from_dict(titles_dict)

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(titles["mod_title"])



In [8]:
def search(query,vectorizer=vectorizer):
    processed = re.sub("[^a-zA-Z0-9 ]", "", query.lower())
    query_vec = vectorizer.transform([query])
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -10)[-10:]
    results = titles.iloc[indices]
    results = results.sort_values("ratings", ascending=False)
    
    return results.head(10)

In [9]:
def get_recs(df = ratings, tb = titles, num_of_rec = 5, rated_book = [], user_id = -1):
  _, best_svd = dump.load("/content/drive/MyDrive/ai/data/best_svd_x")
  new_ratings_df = ratings
  if user_id == -1:
    user_id = ratings.user_id.nunique()+1
    rating_list = []
    for rec in rated_book:
      rated_one_book = {"user_id": user_id, "book_id": rec[0],'rating': rec[1]}
      rating_list.append(rated_one_book)

    df = pd.DataFrame(rating_list)
    new_ratings_df = ratings.append(df, ignore_index = True)
    reader = Reader(rating_scale=(0, 5))
    new_data = Dataset.load_from_df(new_ratings_df[["user_id", 'book_id', 'rating']], reader)
    best_svd.fit(new_data.build_full_trainset())
  list_of_books = []
  for book_id in new_ratings_df["book_id"].unique():
    list_of_books.append((book_id, best_svd.predict(user_id, book_id)[3]))
  ranked_predictions = sorted(list_of_books, key=lambda x:x[1], reverse=True)
  rs = []
  for idx, rec in enumerate(ranked_predictions):
    tl = tb.loc[tb['book_id'] == rec[0].astype(str)]
    rs.append(tl.to_dict("records")[0])
    num_of_rec -=1
    if num_of_rec == 0:
      break
  return rs

In [10]:
like_books = pd.read_csv("/content/drive/MyDrive/ai/data/liked_books.csv", index_col = 0)
like_books = like_books.to_numpy()
arr=like_books[:, 1:3]

In [15]:
class DK():
  def get(self):
    books_recs = get_recs(user_id=282)
    return books_recs
  def get2(self):
    books_recs = get_recs(rated_book=arr)
    return books_recs
dk = DK()
dk.get2()

[{'book_id': '3',
  'cover_image': 'https://images.gr-assets.com/books/1474154022m/3.jpg',
  'mod_title': 'harry potter and the sorcerers stone harry potter 1',
  'mode_title': 'harry potter and the sorcerers stone harry potter 1',
  'ratings': 4765497,
  'title': "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
  'url': 'https://www.goodreads.com/book/show/3.Harry_Potter_and_the_Sorcerer_s_Stone'},
 {'book_id': '6',
  'cover_image': 'https://images.gr-assets.com/books/1361482611m/6.jpg',
  'mod_title': 'harry potter and the goblet of fire harry potter 4',
  'mode_title': 'harry potter and the goblet of fire harry potter 4',
  'ratings': 1792561,
  'title': 'Harry Potter and the Goblet of Fire (Harry Potter, #4)',
  'url': 'https://www.goodreads.com/book/show/6.Harry_Potter_and_the_Goblet_of_Fire'},
 {'book_id': '5',
  'cover_image': 'https://images.gr-assets.com/books/1499277281m/5.jpg',
  'mod_title': 'harry potter and the prisoner of azkaban harry potter 3',
  'mode_title

In [ ]:
app = flask.Flask(__name__)
api = flask_restful.Api(app)

In [ ]:
class GetBook(flask_restful.Resource):
  def get(self):
    books_get = titles.sample(n=10).to_dict("records")
    return books_get

class Hello(flask_restful.Resource):
  def get(self):
    return {"msg": "hello"}
  def post(self):
    json_data = flask.request.get_json(force=True)
    return flask.jsonify({"data": json_data})

In [ ]:
api.add_resource(Hello, '/')
api.add_resource(GetBook, '/getbook')

NgrokTunnel: "http://f84a-35-189-181-119.ngrok.io" -> "http://localhost:5000"


In [ ]:
ngrok.set_auth_token("2AthS7rha8i9A7i0Wms36JzsMhz_5ZEnF1HLDn67xshoaKRgw")
public_url = ngrok.connect(5000)
print(public_url)
import threading
import os
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()